In [ ]:
# %matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
import os 
DATA_DIR = "/home/tlips/Code/few-shot-keypoints/data/RPL-clean-pen/3mark_1arm_fullresolution"


demo_dirs = os.listdir(DATA_DIR)
# filter out directories that are not demos
demo_dirs = [d for d in demo_dirs if os.path.isdir(os.path.join(DATA_DIR, d))]
# sort by name
demo_dirs.sort()
print(f"Found {len(demo_dirs)} demonstrations")


# ðemo dir 
#  back_camera
# data.mp4

video_paths = [os.path.join(DATA_DIR, demo_dir, "back_camera", "data.mp4") for demo_dir in demo_dirs]
print(video_paths)

In [ ]:
# get initial frames for each video

initial_frames = []
for video_path in video_paths:
    video = cv2.VideoCapture(video_path)
    ret, frame = video.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    initial_frames.append(frame)

# save the initial frames
initial_frames = np.array(initial_frames)

In [ ]:
# # instead, now get all frames of a single video
url = "/home/tlips/Code/few-shot-keypoints/data/RPL-clean-pen/3mark_1arm_fullresolution/demo_1/back_camera/data.mp4"

import imageio.v3 as iio
frames = iio.imread(url, plugin="FFMPEG")  # plugin="pyav"
# frames = frames[-1100:-800]
print(len(frames))
initial_frames = frames[::40]
print(len(initial_frames))

In [ ]:
from transformers import AutoModelForZeroShotObjectDetection, AutoProcessor
model_id =  "IDEA-Research/grounding-dino-tiny"
# model_id = "google/owlv2-base-patch16"
device = "cuda"
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)
processor = AutoProcessor.from_pretrained(model_id)









In [ ]:
import torch


# will only compile on first call
# img = np.random.randint(0, 255, (100, 100, 3))
# text_labels = ["dummy"]
# inputs = processor(text=text_labels, images=img, return_tensors="pt").to(device)


# model.compile()


# with torch.no_grad():
#     outputs = model(**inputs)


In [ ]:
text_labels = ["pen", "box"]


In [ ]:
import torch
color_list = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255), (255, 255, 255), (0, 0, 0)]


img = initial_frames[0]

print(img.shape)


def get_bounding_boxes(img, text_labels):
    # https://github.com/huggingface/transformers/blob/main/src/transformers/models/grounding_dino/processing_grounding_dino.py
    
    text_labels = [text_labels]
    inputs = processor(text=text_labels, images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    results = processor.post_process_grounded_object_detection(outputs, inputs.input_ids,threshold=0.3, target_sizes=[img.shape[:2]])
    return results[0]






results = get_bounding_boxes(img, text_labels)



In [ ]:
def visualize_boxes_on_image(img, results, text_labels):
    annotated_img = img.copy()
    # check if any annotations
    if len(results['boxes']) is None or len(results['boxes']) == 0:
        return annotated_img
    for box, label, score in zip(results['boxes'], results['labels'], results['scores']):
    
        x1, y1, x2, y2 = box
        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        label_idx = text_labels.index(label)
        annotated_img = cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color_list[label_idx], 2)
        annotated_img = cv2.putText(annotated_img, f"{label}: {score:.2f}", (x1, y2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    return annotated_img


annotated_img = visualize_boxes_on_image(img, results, text_labels)
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

In [ ]:
# run the model on all frames and store the results in a list 

initial_frame_results  =  []
times = []
import time 
for frame in initial_frames:
    start_time = time.time()
    # resize frame to 256x256
    # frame = cv2.resize(frame, (256, 256))
    results = get_bounding_boxes(frame, text_labels)
    end_time = time.time()
    times.append(end_time - start_time)
    initial_frame_results.append(results)


print(f"Average time per frame: {sum(times) / len(times)}")



In [ ]:
# visualize in a single figure with 5 by len(initial_frames) // 5 + 1 grid

fig, axs = plt.subplots(len(initial_frames) // 5 + 1, 5, figsize=(25, 5 * len(initial_frames) // 5))
for i, frame in enumerate(initial_frames):
    # frame = cv2.resize(frame, (256, 256))
    img = visualize_boxes_on_image(frame, initial_frame_results[i], text_labels)
    axs[i // 5, i % 5].imshow(img)
    axs[i // 5, i % 5].axis("off")
    axs[i // 5, i % 5].set_title(f" {i}")
plt.show()